# CityBikes

Send a request to CityBikes for the city of your choice. 

In [41]:
#create an api request to citybik.es to get the information from Hamilton Ontario
import requests
import json
import pandas as pd

In [42]:
#make the request
response = requests.get('https://api.citybik.es/v2/networks')

#convert the response to a json object
data = response.json()

#convert the json object to a pandas dataframe
citybikes_df = pd.DataFrame(data['networks'])

In [43]:
citybikes_df.head()

,company,href,id,location,name,source,gbfs_href,license,ebikes
0,[ЗАО «СитиБайк»],/v2/networks/velobike-moscow,velobike-moscow,"{'city': 'Moscow', 'country': 'RU', 'latitude'...",Velobike,NaN,NaN,NaN,NaN
1,[Comunicare S.r.l.],/v2/networks/bicincitta-siena,bicincitta-siena,"{'city': 'Siena', 'country': 'IT', 'latitude':...",Bicincittà,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN
2,[Cyclopolis Systems],/v2/networks/cyclopolis-maroussi,cyclopolis-maroussi,"{'city': 'Maroussi', 'country': 'GR', 'latitud...",Cyclopolis,NaN,NaN,NaN,NaN
3,[Cyclopolis Systems],/v2/networks/cyclopolis-nafplio,cyclopolis-nafplio,"{'city': 'Nafplio', 'country': 'GR', 'latitude...",Cyclopolis,NaN,NaN,NaN,NaN
4,[Comunicare S.r.l.],/v2/networks/bicincitta-parco-dei-colli-di-ber...,bicincitta-parco-dei-colli-di-bergamo,"{'city': 'Parco dei Colli di Bergamo', 'countr...",Bicincittà,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN


In [44]:
#json normalization of the location column
location = pd.json_normalize(citybikes_df['location'])
location.columns = ['location.' + str(col) for col in location.columns]
citybikes_df = citybikes_df.drop('location', axis=1)
citybikes_df = pd.concat([citybikes_df, location], axis=1)


In [45]:
citybikes_df.head()

,company,href,id,name,source,gbfs_href,license,ebikes,location.city,location.country,location.latitude,location.longitude
0,[ЗАО «СитиБайк»],/v2/networks/velobike-moscow,velobike-moscow,Velobike,NaN,NaN,NaN,NaN,Moscow,RU,55.750000,37.616667
1,[Comunicare S.r.l.],/v2/networks/bicincitta-siena,bicincitta-siena,Bicincittà,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN,Siena,IT,43.318600,11.330600
2,[Cyclopolis Systems],/v2/networks/cyclopolis-maroussi,cyclopolis-maroussi,Cyclopolis,NaN,NaN,NaN,NaN,Maroussi,GR,38.056872,23.808330
3,[Cyclopolis Systems],/v2/networks/cyclopolis-nafplio,cyclopolis-nafplio,Cyclopolis,NaN,NaN,NaN,NaN,Nafplio,GR,37.563940,22.809340
4,[Comunicare S.r.l.],/v2/networks/bicincitta-parco-dei-colli-di-ber...,bicincitta-parco-dei-colli-di-bergamo,Bicincittà,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN,Parco dei Colli di Bergamo,IT,45.722956,9.649230


In [46]:
#filter the dataframe to only show the information for 'Hamilton, ON' in the location.city column
hamilton_bike_companies = citybikes_df[citybikes_df['location.city'] == 'Hamilton, ON']
hamilton_bike_companies.head()

,company,href,id,name,source,gbfs_href,license,ebikes,location.city,location.country,location.latitude,location.longitude
74,[Social Bicycles Inc.],/v2/networks/sobi-hamilton,sobi-hamilton,SoBi,NaN,https://hamilton.socialbicycles.com/opendata/g...,NaN,NaN,"Hamilton, ON",CA,43.256436,-79.869297


In [47]:
#create a new request to get the information for the Hamilton Social Bicycles inc. api using the link from the gbfs_href column
#its suprising to me that there is only one company in Hamilton for bike share
response = requests.get(hamilton_bike_companies['gbfs_href'].values[0])
data = response.json()
sobi_df = pd.DataFrame(data['data']['en']['feeds'])
sobi_df

,name,url
0,gbfs,http://hamilton.socialbicycles.com/opendata/gb...
1,system_information,http://hamilton.socialbicycles.com/opendata/sy...
2,station_information,http://hamilton.socialbicycles.com/opendata/st...
3,station_status,http://hamilton.socialbicycles.com/opendata/st...
4,free_bike_status,http://hamilton.socialbicycles.com/opendata/fr...
5,system_hours,http://hamilton.socialbicycles.com/opendata/sy...
6,system_calendar,http://hamilton.socialbicycles.com/opendata/sy...
7,system_regions,http://hamilton.socialbicycles.com/opendata/sy...
8,system_pricing_plans,http://hamilton.socialbicycles.com/opendata/sy...
9,system_alerts,http://hamilton.socialbicycles.com/opendata/sy...


In [48]:
#create a new request to grab station information from the station_information url
response = requests.get(sobi_df[sobi_df['name'] == 'station_information']['url'].values[0])
data = response.json()
station_info = pd.DataFrame(data['data']['stations'])
station_info.head()

,station_id,name,region_id,lon,lat,address,rental_methods
0,hub_435,Gore Park,region_86,-79.868664,43.256317,"20 King Street East, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU..."
1,hub_436,James North at Mulberry,region_86,-79.866743,43.261819,"200 James Street North, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU..."
2,hub_527,Seedworks,region_86,-79.863613,43.259111,"126 Catharine Street North, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU..."
3,hub_530,Cootes at York and King,region_86,-79.949656,43.265581,"10 Cootes Drive, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU..."
4,hub_535,Osler at Grant,region_86,-79.942643,43.259550,"89-101 Osler Drive, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU..."


In [49]:
#create a new request to grab station status information from the station_status url
response = requests.get(sobi_df[sobi_df['name'] == 'station_status']['url'].values[0])
data = response.json()
station_status = pd.DataFrame(data['data']['stations'])
station_status.head()

,station_id,num_bikes_available,num_bikes_disabled,num_docks_available,is_installed,is_renting,is_returning,last_reported
0,hub_435,2,0,14,1,1,1,1709853284
1,hub_436,7,0,11,1,1,1,1709853284
2,hub_527,5,0,2,1,1,1,1709853284
3,hub_530,7,0,7,1,1,1,1709853284
4,hub_535,6,0,6,1,1,1,1709853284


In [50]:
#create a new request to grab free bike information from the free_bike_status url
response = requests.get(sobi_df[sobi_df['name'] == 'free_bike_status']['url'].values[0])
data = response.json()
free_bike_status = pd.DataFrame(data['data']['bikes'])
free_bike_status.head()


,bike_id,name,lon,lat,is_reserved,is_disabled,jump_ebike_battery_level
0,bike_920,092-Turn Down For What?,-79.918028,43.263007,0,0,NaN
1,bike_38506,908-The Meg,-79.830162,43.257590,0,0,NaN
2,bike_2015,254-Dallas,-79.849608,43.264480,0,0,NaN
3,bike_3138,636-Kareem,-79.924270,43.254348,0,0,NaN
4,bike_2032,272-Gilda,-79.916702,43.265388,0,0,NaN


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [51]:
#join the station_info and station_status dataframes on the station_id column
sobi_station_info = pd.merge(station_info, station_status, on='station_id')
sobi_station_info.head()

,station_id,name,region_id,lon,lat,address,rental_methods,num_bikes_available,num_bikes_disabled,num_docks_available,is_installed,is_renting,is_returning,last_reported
0,hub_435,Gore Park,region_86,-79.868664,43.256317,"20 King Street East, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU...",2,0,14,1,1,1,1709853284
1,hub_436,James North at Mulberry,region_86,-79.866743,43.261819,"200 James Street North, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU...",7,0,11,1,1,1,1709853284
2,hub_527,Seedworks,region_86,-79.863613,43.259111,"126 Catharine Street North, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU...",5,0,2,1,1,1,1709853284
3,hub_530,Cootes at York and King,region_86,-79.949656,43.265581,"10 Cootes Drive, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU...",7,0,7,1,1,1,1709853284
4,hub_535,Osler at Grant,region_86,-79.942643,43.259550,"89-101 Osler Drive, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU...",6,0,6,1,1,1,1709853284


In [52]:
#filter the sobi_station_info dataframe to only show the stations that have bikes available and save the result to a new dataframe
available_bikes = sobi_station_info[sobi_station_info['num_bikes_available'] > 0]
available_bikes.head()

,station_id,name,region_id,lon,lat,address,rental_methods,num_bikes_available,num_bikes_disabled,num_docks_available,is_installed,is_renting,is_returning,last_reported
0,hub_435,Gore Park,region_86,-79.868664,43.256317,"20 King Street East, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU...",2,0,14,1,1,1,1709853284
1,hub_436,James North at Mulberry,region_86,-79.866743,43.261819,"200 James Street North, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU...",7,0,11,1,1,1,1709853284
2,hub_527,Seedworks,region_86,-79.863613,43.259111,"126 Catharine Street North, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU...",5,0,2,1,1,1,1709853284
3,hub_530,Cootes at York and King,region_86,-79.949656,43.265581,"10 Cootes Drive, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU...",7,0,7,1,1,1,1709853284
4,hub_535,Osler at Grant,region_86,-79.942643,43.259550,"89-101 Osler Drive, Hamilton","[KEY, APPLEPAY, ANDROIDPAY, TRANSITCARD, ACCOU...",6,0,6,1,1,1,1709853284


In [53]:
#remove region_id , is_installed, is_renting, is_returning, rental_methods, and num_docks_available columns from the available_bikes dataframe
available_bikes = available_bikes.drop(['region_id', 'is_installed', 'is_renting', 'is_returning','rental_methods', 'num_docks_available'], axis=1)
available_bikes.head()

,station_id,name,lon,lat,address,num_bikes_available,num_bikes_disabled,last_reported
0,hub_435,Gore Park,-79.868664,43.256317,"20 King Street East, Hamilton",2,0,1709853284
1,hub_436,James North at Mulberry,-79.866743,43.261819,"200 James Street North, Hamilton",7,0,1709853284
2,hub_527,Seedworks,-79.863613,43.259111,"126 Catharine Street North, Hamilton",5,0,1709853284
3,hub_530,Cootes at York and King,-79.949656,43.265581,"10 Cootes Drive, Hamilton",7,0,1709853284
4,hub_535,Osler at Grant,-79.942643,43.259550,"89-101 Osler Drive, Hamilton",6,0,1709853284


In [54]:
#update the last_reported column to be a datetime object
available_bikes['last_reported'] = pd.to_datetime(available_bikes['last_reported'], unit='s')
available_bikes.head()

,station_id,name,lon,lat,address,num_bikes_available,num_bikes_disabled,last_reported
0,hub_435,Gore Park,-79.868664,43.256317,"20 King Street East, Hamilton",2,0,2024-03-07 23:14:44
1,hub_436,James North at Mulberry,-79.866743,43.261819,"200 James Street North, Hamilton",7,0,2024-03-07 23:14:44
2,hub_527,Seedworks,-79.863613,43.259111,"126 Catharine Street North, Hamilton",5,0,2024-03-07 23:14:44
3,hub_530,Cootes at York and King,-79.949656,43.265581,"10 Cootes Drive, Hamilton",7,0,2024-03-07 23:14:44
4,hub_535,Osler at Grant,-79.942643,43.259550,"89-101 Osler Drive, Hamilton",6,0,2024-03-07 23:14:44


Put your parsed results into a DataFrame.

In [55]:
available_bikes

,station_id,name,lon,lat,address,num_bikes_available,num_bikes_disabled,last_reported
0,hub_435,Gore Park,-79.868664,43.256317,"20 King Street East, Hamilton",2,0,2024-03-07 23:14:44
1,hub_436,James North at Mulberry,-79.866743,43.261819,"200 James Street North, Hamilton",7,0,2024-03-07 23:14:44
2,hub_527,Seedworks,-79.863613,43.259111,"126 Catharine Street North, Hamilton",5,0,2024-03-07 23:14:44
3,hub_530,Cootes at York and King,-79.949656,43.265581,"10 Cootes Drive, Hamilton",7,0,2024-03-07 23:14:44
4,hub_535,Osler at Grant,-79.942643,43.259550,"89-101 Osler Drive, Hamilton",6,0,2024-03-07 23:14:44
...,...,...,...,...,...,...,...,...
138,hub_5845,Jackson Square,-79.869577,43.256935,"2-44 King Street West, Hamilton",10,0,2024-03-07 23:14:44
139,hub_5846,Cannon at Steven,-79.849981,43.256461,"400-408 Cannon Street East, Hamilton",8,0,2024-03-07 23:14:44
141,hub_5903,Pier 8,-79.856907,43.275289,"595 Catharine Street North, Hamilton",2,0,2024-03-07 23:14:44
142,hub_6148,King William at James,-79.868487,43.257635,"39 King William Street, Hamilton",2,0,2024-03-07 23:14:44


In [56]:
#order the available_bikes dataframe by the num_bikes_available column in descending order
available_bikes = available_bikes.sort_values(by='num_bikes_available', ascending=False)
available_bikes

,station_id,name,lon,lat,address,num_bikes_available,num_bikes_disabled,last_reported
112,hub_2570,Athletic Centre Bike Racks,-79.915746,43.264715,"Stearn Drive, McMaster University, Hamilton",22,0,2024-03-07 23:14:44
81,hub_624,McMaster Health Sciences,-79.918248,43.260531,"CIBC, University Avenue, Westdale, Hamilton, O...",18,0,2024-03-07 23:14:44
83,hub_626,McMaster Stadium,-79.918681,43.264622,"Stearn Drive, McMaster University, Hamilton",16,0,2024-03-07 23:14:44
51,hub_583,Discovery Drive,-79.860298,43.276067,"Waterfront Trail, Hamilton",14,0,2024-03-07 23:14:44
20,hub_551,Longwood at Edgevale,-79.899309,43.266211,"78, Longwood Road North, Westdale, Hamilton, O...",13,0,2024-03-07 23:14:44
...,...,...,...,...,...,...,...,...
96,hub_867,Queen at George,-79.879358,43.258122,"34-54 Queen Street South, Hamilton",1,0,2024-03-07 23:14:44
127,hub_3885,George Street,-79.878950,43.258514,"118, George Street, Hess Village, Hamilton, On...",1,0,2024-03-07 23:14:44
98,hub_903,Duke at James,-79.871354,43.251630,"180 James Street South, Hamilton",1,0,2024-03-07 23:14:44
123,hub_2923,Dunsmure at Sherman - ERI10,-79.838888,43.249433,"44 Dunsmure Road, Hamilton",1,0,2024-03-07 23:14:44
